# Tutorial: Benzene

## Introduction
In this tutorial, we'll focus on generating integrals for the PPP model considering real organic molecule --  Benzene.
This example is used to showcase the simplicity of usage a ModelHamiltoian package to build a Hamiltonian that corresponds to the one found in the [literature](https://www.sciencedirect.com/science/article/pii/S0301010499000178). 

To do this, we are going to consider the most general occupation-number Hamiltonian we consider is the generalized Pariser-Parr-Pople (PPP) Hamiltonian:

$$\hat{H}_{\text{PPP}} = \sum_{pq} h_{pq} a_p^\dagger a_q + \sum_p U_p \hat{n}_{p\alpha}\hat{n}_{p\beta} + \frac{1}{2}\sum_{p\ne q} \gamma_{pq} (\hat{n}_{p \alpha} + \hat{n}_{p \beta} - Q_p)(\hat{n}_{q \alpha} + \hat{n}_{q \beta} - Q_q) $$


## Example: Defining PPP Hamiltonian

There are several ways to build a PPP Hamiltonian. The most straightforward way is to evaluate $gamma$ matrix and provide it to the `ModelHamiltonian` class. 


In [1]:
# import sys 
# sys.path.insert(0, '../')
import numpy as np 
import sys 
import numpy as np 
from moha import HamPPP 
from pyscf import gto, scf, fci 

def build_gamma(norb=6):
    # Define the gamma matrix here (example shape) 
    gamma_matrix = np.zeros((norb, norb))

    gamma_a_a1 = 5.27760 
    gamma_a_a2 = 3.86206 
    gamma_a_a3 = 3.48785 
    
    # Fill the matrix according to the given rules 
    for a in range(norb): 
        gamma_matrix[a, a-1] = gamma_a_a1 
        gamma_matrix[a, a-2] = gamma_a_a2 
        gamma_matrix[a, a-3] = gamma_a_a3 
        
    gamma_matrix = np.maximum(gamma_matrix, np.transpose(gamma_matrix))
    return gamma_matrix
 
# Define the benzene system with a 6-membered ring structure 
system = [('C1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'C1', 1)] 


#Define U_onsite, alpha and gamma matrix 
u_onsite = 10.84 * np.ones(6) / 2
alpha = 0 
 
gamma_matrix = build_gamma()
# Loop to vary beta from 0 to -5
for beta in np.linspace(0, -5, num=6):
    # Generate the PPP object
    ppp_hamiltonian = HamPPP(system, alpha=alpha, beta=beta, u_onsite=u_onsite, gamma=gamma_matrix, charges=np.ones(6))
    
    # Generate the 0, 1, 2 body integral elements
    e01 = ppp_hamiltonian.generate_zero_body_integral() 
    h11 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
    h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis', sym=4) 
 
    # Convert h2 to chemists notation 
    h21_ch = 1*np.transpose(h2, (0, 2, 1, 3)) 
    norb = h11.shape[0] 
    nelec = norb  # Assuming one electron per site  

    # Perform the FCI calculation 
    e_fci, ci = fci.direct_spin1.kernel(1*h11, 2*h21_ch, norb, nelec, ecore=e01, nroots=1)
    print(f'Beta: {beta}, FCI Energy: {e_fci}')


Beta: 0.0, FCI Energy: -1.4210854715202004e-14
Beta: -1.0, FCI Energy: -2.779514577625207
Beta: -2.0, FCI Energy: -9.084603708594514
Beta: -3.0, FCI Energy: -16.484889549995387
Beta: -4.0, FCI Energy: -24.195014064799068
Beta: -5.0, FCI Energy: -32.024692914537454


# PPP Benzene: Building gamma

What if we want to build $gamma$ matrix for a molecule? The power and flexibility of the `ModelHamiltonian` package allows us to do that.


MoHa package offers a functionality to compute $gamma$ using the module `PariserParr` and the function `compute_gamma`. 
Computation of $gamma$ matrix requires two ingridients: 
* `u_onsite` - potential on each site/atoms
* `R_xy` - distances between sites/atoms

Note: you don't need to provide any of those parameters. If `u_onsite` is not provided, the matrix $\overline{U}_{XY}$ will be populated using the formula:
$$
\overline{U}_{XY} = \frac{1}{2}(U_X + U_Y)
$$ 

where 
$$ U_x = I_x - A_x $$
Values of `I_x` and `A_x` ionisation potential and electron affinity of the atom `x` respectively. User can provide those values in the form of a dictionary `affinity_dct` and `atom_dict` or use the default values.


If `R_xy` is not provided, it will be automatically populated from the distances provided at `connectivity` parameter.



Finally, $gamma$ is computed using the formula:

$$
\gamma_{XY} = \frac{\overline{U}_{XY}}{\overline{U}_{XY} R_{XY} + e^{-\frac{1}{2} \overline{U}_{XY}^2 R_{XY}^2}}
$$







Defining the Benzene system following th same structure defined in the previous tutorials, we decide to move on with Full Configuration Iteraction of the hamiltonian, building $gamma$ matrix using the `PariserParr` module.

Varying $\beta$ from 0 to -5, in this piece of code we prove that we can achieve the spectrum of energies reported in Bendazzoli et al.

In [2]:
from moha.rauk import PariserParr

def build_Rxy_matrix(norb=6):
    d = np.zeros((norb, norb))
    for mu in range(6):
        for nu in range(6):
            d[mu, nu] = 1.4 * np.sin(np.pi * abs(mu - nu) / norb) / np.sin(np.pi / norb)
    return d

# Define the benzene system with a 6-membered ring structure
system = [('C1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'C1', 1)] 

Rxy_matrix = build_Rxy_matrix()

# compute u_onsite for the given system
u, _ = PariserParr.compute_u(system, ionization_dictionary={'C': 11.16}, # ionization energy of carbon in eV
                                     affinity_dictionary={'C': 0.03}) # electron affinity of carbon in eV

# compute gamma for the given system
gamma_matrix_ppp = PariserParr.compute_gamma(system, Rxy_matrix=Rxy_matrix,
                                             u_onsite=u/2)

alpha = 0

# Loop to vary beta from 0 to -5
for beta in np.linspace(0, -5, num=6):
    ppp_hamiltonian = HamPPP(system, alpha=alpha, gamma = gamma_matrix_ppp, beta=beta, u_onsite=u/2, charges=np.ones(6))
    
    # Generate the 0, 1, 2 body integral elements
    e0 = ppp_hamiltonian.generate_zero_body_integral() 
    h1 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
    h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)
    
    # Convert h2 to chemists notation
    h2_ch = np.transpose(h2, (0, 2, 1, 3))
    norb = h1.shape[0]
    nelec = norb  # Assuming one electron per site  

    # Perform the FCI calculation
    e_fci, ci = fci.direct_spin1.kernel(1*h1, 2*h2_ch, norb, nelec, ecore=e0, nroots=1)
    print(f'Beta: {beta}, FCI Energy: {e_fci}')

Beta: 0.0, FCI Energy: 8.881784197001252e-16
Beta: -1.0, FCI Energy: -1.6000550795826625
Beta: -2.0, FCI Energy: -5.956135566002675
Beta: -3.0, FCI Energy: -12.105432809022926
Beta: -4.0, FCI Energy: -19.126580752154776
Beta: -5.0, FCI Energy: -26.54429608422473


## Heteroatoms

The ModelHamiotonian package also supports the possibility of using different type of atoms, using the Rauk’s table (RAUK, 2001).

To do this, simply replace an atom of C with one of B.


In [3]:
import sys 
import numpy as np 
from moha import HamPPP 
from pyscf import gto, scf, fci 
 
# Define the benzene system with a 6-membered ring structure 
system = [('B1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'B1', 1)] 
 
u_onsite = np.array([10.84, 10.84, 10.84, 10.84, 10.84, 10.84]) / 2
gamma_matrix = PariserParr.compute_gamma(system, u_onsite=u_onsite)

norb = 6  # Number of orbitals for benzene (6 carbons) 
ppp_hamiltonian = HamPPP(system, u_onsite=u_onsite, gamma=gamma_matrix, charges=np.array([1 for _ in range(6)]))

e0 = ppp_hamiltonian.generate_zero_body_integral() 
h1 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis', sym=4) 
 
# Convert h2 to chemists notation 
h2_ch = np.transpose(h2, (0, 2, 1, 3)) 
norb = h1.shape[0] 
nelec = norb  # Assuming one electron per site  

# Perform the FCI calculation 
e_fci, ci = fci.direct_spin1.kernel(h1, h2_ch, norb, nelec, ecore=e0, nroots=1)
print(f'Alpha: {ppp_hamiltonian.alpha}, Beta: {ppp_hamiltonian.beta}, FCI Energy: {e_fci}')


Alpha: -0.414, Beta: -0.0533, FCI Energy: -29.85842553617428


# References

Bendazzoli, G. L., Evangelisti, S., & Gagliardi, L. (1994). Full configuration interaction study of the ground state of closed-shell cyclic PPP polyenes. International Journal of Quantum Chemistry, 51(1), 117-123. https://doi.org/10.1002/qua.560510104



